<a href="https://colab.research.google.com/github/lauramartinho/Underwater-image-enhancement-based-on-fusion-of-intensity-transformation-techniques/blob/main/UIEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Imports***

In [1]:
import cv2 #OpenCv cv2
from google.colab.patches import cv2_imshow
import numpy as np #Numpy
from google.colab import files 
import os
from scipy import ndimage
import math
from PIL import Image, ImageOps
from glob import glob
from os.path import join
from ntpath import basename
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import sys
from skimage import io, color, filters, measure
from scipy.ndimage import gaussian_filter

### ***Image Processing Pipeline - functions***

In [2]:
def colorCorrection(imagem, intensidade):
  resultados = [] #vetor para receber os resultados das trasnformações 
  rgb = cv2.split(imagem) #acesso a cada canal de cor
  saturacao = rgb[0].shape[0] * rgb[0].shape[1] * intensidade / 500.0 #200
  for canal in rgb:
      histograma = cv2.calcHist([canal], [0], None, [256], (0,256), accumulate=False)
      #low value
      lowvalue = np.searchsorted(np.cumsum(histograma), saturacao) #soma acumulada dos elementos valor inferior do histograma e encontra índices onde os elementos devem ser inseridos p/ ordenar
      #high value
      highvalue = 255-np.searchsorted(np.cumsum(histograma[::-1]), saturacao)#soma acumulada e sort valores superiores
      #tomar toda a informação (min/max) da curva linear para aplicar e gerar uma LUT de 256 valores a aplicar nos canais stretching
      lut = np.array([0 if i < lowvalue else (255 if i > highvalue else round(float(i-lowvalue)/float(highvalue-lowvalue)*255)) for i in np.arange(0, 256)], dtype="uint8")
      #mescla os canais de volta
      resultados.append(cv2.LUT(canal, lut))
  return cv2.merge(resultados)

In [3]:
def gammaCorrection(image, gamma=0.8):  #0.7
#construir uma tabela de novos valores mapeados de pixel [0, 255] para seus valores gama ajustados
  invGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** invGamma) * 255
  for i in np.arange(0, 256)]).astype("uint8")
  #aplicar a correção 
  return cv2.LUT(image, table)

In [4]:
def mascaraNitidez(imagem, kernel=(5, 5), sigma=0.5, intensidade=1.0, threshold=0): #sigma 1.0, intensidade 2.0
  suavizacao = cv2.GaussianBlur(imagem, kernel, sigma)
  nitidez = float(intensidade + 1) * imagem - float(intensidade) * suavizacao
  nitidez = np.maximum(nitidez, np.zeros(nitidez.shape))
  nitidez = np.minimum(nitidez, 255 * np.ones(nitidez.shape))
  nitidez = nitidez.round().astype(np.uint8)
  if threshold > 0:
      contraste_baixo = np.absolute(imagem - suavizacao) < threshold
      np.copyto(nitidez, imagem, where=contraste_baixo)
  return nitidez

In [5]:
def CLAHE(imagem):
  clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(4, 4))
  for i in range(3):
    imagem[:, :, i] = clahe.apply((imagem[:, :, i]))
  return imagem

In [6]:
def add(a ,b):
  fusao = cv2.addWeighted(a, 0.8, b, 0.5, 0) #combina duas imagens
  #0.5/0.5 valores alfa e beta
  return fusao

def enhacement(imagem):
  brightness = 5
  contrast = 10
  img = np.int16(imagem)
  img = img * (contrast/127+1) - contrast + brightness
  img = np.clip(img, 0, 255)
  final = np.uint8(img)
  return final

In [7]:
def compare(a,b):
   input = a
   output= b
   cp = np.hstack((input, output))
   return cv2_imshow(cp)

### ***Main Function***

In [8]:
def UWE(image):
   
  colorCorrected = colorCorrection(image, 1.0)
  gammaCorrected = gammaCorrection(colorCorrected)
  edgeEnhacement = mascaraNitidez(gammaCorrected)
  clahe = CLAHE(colorCorrected)
  gammaCorrected2 = gammaCorrection(colorCorrected)
  edgeEnhacement2 = mascaraNitidez(gammaCorrected2)
  fusao = add(edgeEnhacement, edgeEnhacement2)
 
  output = enhacement(fusao)
  
  return output

### ***Evaluation***

#### ***UIQM***
* non reference underwater image quality measure (UIQM) 
* Attribute measures:
colourfulness measure (UICM); sharpness measure (UISM); contrast measure (UIConM).

##### ***Assist functions***

In [10]:
def mu_a(x, alpha_L=0.1, alpha_R=0.1):
 
    # Calculates the asymetric alpha-trimmed 
    # Média aparada (dispersão) 

    # sort pixels by intensity - for clipping
    x = sorted(x)
    # get number of pixels
    K = len(x)
    # calculate T alpha L and T alpha R
    T_a_L = math.ceil(alpha_L * K)
    T_a_R = math.floor(alpha_R * K)
    # calculate mu_alpha weight
    weight = (1 / (K - T_a_L - T_a_R))
    # loop through flattened image starting at T_a_L+1 and ending at K-T_a_R
    s = int(T_a_L + 1)
    e = int(K - T_a_R)
    val = sum(x[s:e])
    val = weight * val
    return val


def s_a(x, mu):
    val = 0
    for pixel in x:
        val += math.pow((pixel - mu), 2)
    return val / len(x)

def sobel(x):
    dx = ndimage.sobel(x, 0)
    dy = ndimage.sobel(x, 1)
    mag = np.hypot(dx, dy)
    mag *= 255.0 / np.max(mag)
    return mag


def eme(x, window_size):
      #Enhancement measure estimation
      #x.shape[0] = height
      #x.shape[1] = width
    
    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1] // window_size
    k2 = x.shape[0] // window_size

    # weight
    w = 2. / (k1 * k2)

    blocksize_x = window_size
    blocksize_y = window_size

    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[:blocksize_y * k2, :blocksize_x * k1]

    val = 0
    for l in range(k1):
        for k in range(k2):
            block = x[k * window_size:window_size * (k + 1), l * window_size:window_size * (l + 1)]
            max_ = np.max(block)
            min_ = np.min(block)

            # bound checks, can't do log(0)
            if min_ == 0.0:
                val += 0
            elif max_ == 0.0:
                val += 0
            else:
                val += math.log(max_ / min_)
    return w * val


###########################################
#logAMEE
###########################################
def plip_g(x, mu=1026.0):
    return mu - x


def plip_theta(g1, g2, k):
    g1 = plip_g(g1)
    g2 = plip_g(g2)
    return k * ((g1 - g2) / (k - g2))


def plip_cross(g1, g2, gamma):
    g1 = plip_g(g1)
    g2 = plip_g(g2)
    return g1 + g2 - ((g1 * g2) / (gamma))


def plip_diag(c, g, gamma):
    g = plip_g(g)
    return gamma - (gamma * math.pow((1 - (g / gamma)), c))


def plip_multiplication(g1, g2):
    return plip_phiInverse(plip_phi(g1) * plip_phi(g2))
    # return plip_phiInverse(plip_phi(plip_g(g1)) * plip_phi(plip_g(g2)))


def plip_phiInverse(g):
    plip_lambda = 1026.0
    plip_beta = 1.0
    return plip_lambda * (1 - math.pow(math.exp(-g / plip_lambda), 1 / plip_beta));


def plip_phi(g):
    plip_lambda = 1026.0
    plip_beta = 1.0
    return -plip_lambda * math.pow(math.log(1 - g / plip_lambda), plip_beta)

##### ***UICM, UISM, UICONM***

In [11]:
def _uicm(x):
    R = x[:, :, 0].flatten()
    G = x[:, :, 1].flatten()
    B = x[:, :, 2].flatten()
    RG = R - G
    YB = ((R + G) / 2) - B
    mu_a_RG = mu_a(RG)
    mu_a_YB = mu_a(YB)
    s_a_RG = s_a(RG, mu_a_RG)
    s_a_YB = s_a(YB, mu_a_YB)
    l = math.sqrt((math.pow(mu_a_RG, 2) + math.pow(mu_a_YB, 2)))
    r = math.sqrt(s_a_RG + s_a_YB)
    return (-0.0268 * l) + (0.1586 * r)


def _uism(x):
    # get image channels
    R = x[:, :, 0]
    G = x[:, :, 1]
    B = x[:, :, 2]

    # first apply Sobel edge detector to each RGB component
    Rs = sobel(R)
    Gs = sobel(G)
    Bs = sobel(B)

    # multiply the edges detected for each channel by the channel itself
    R_edge_map = np.multiply(Rs, R)
    G_edge_map = np.multiply(Gs, G)
    B_edge_map = np.multiply(Bs, B)

    # get eme for each channel
    r_eme = eme(R_edge_map, 8)
    g_eme = eme(G_edge_map, 8)
    b_eme = eme(B_edge_map, 8)

    # coefficients
    lambda_r = 0.299
    lambda_g = 0.587
    lambda_b = 0.144

    return (lambda_r * r_eme) + (lambda_g * g_eme) + (lambda_b * b_eme)


def _uiconm(x, window_size):
    plip_lambda = 1026.0
    plip_gamma = 1026.0
    plip_beta = 1.0
    plip_mu = 1026.0
    plip_k = 1026.0

    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1] // window_size
    k2 = x.shape[0] // window_size

    # weight
    w = -1. / (k1 * k2)

    blocksize_x = window_size
    blocksize_y = window_size

    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[:blocksize_y * k2, :blocksize_x * k1]

    # entropy scale - higher helps with randomness
    alpha = 1

    val = 0
    for l in range(k1):
        for k in range(k2):
            block = x[k * window_size:window_size * (k + 1), l * window_size:window_size * (l + 1), :]
            max_ = np.max(block)
            min_ = np.min(block)

            top = max_ - min_
            bot = max_ + min_

            if math.isnan(top) or math.isnan(bot) or bot == 0.0 or top == 0.0:
                val += 0.0
            else:
                val += alpha * math.pow((top / bot), alpha) * math.log(top / bot)

            # try: val += plip_multiplication((top/bot),math.log(top/bot))
    return w * val

##### ***Main Uiqm***

In [12]:
def getUIQM(x):
    """
      Function to return UIQM to be called from other programs
      x: image
    """
    x = x.astype(np.float32)
    ### from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7300447
    # c1 = 0.4680; c2 = 0.2745; c3 = 0.2576
    ### from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7300447
    """
    c1 = 0.0282
    c2 = 0.2953
    c3 = 3.5753
    """
    #"""
    c1 = 0.4680
    c2 = 0.2745
    c3 = 0.2576
    #"""
    uicm = _uicm(x)
    uism = _uism(x)
    uiconm = _uiconm(x, 8)
    uiqm = (c1 * uicm) + (c2 * uism) + (c3 * uiconm)
    return uiqm

#### ***UCIQE***

In [14]:
def getUCIQE(img):
    img_BGR = img
    img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB) 
    img_LAB = np.array(img_LAB,dtype=np.float64)
    # Trained coefficients are c1=0.4680, c2=0.2745, c3=0.2576 according to paper.
    coe_Metric = [0.4680, 0.2745, 0.2576]
    
    img_lum = img_LAB[:,:,0]/255.0
    img_a = img_LAB[:,:,1]/255.0
    img_b = img_LAB[:,:,2]/255.0

    # item-1
    chroma = np.sqrt(np.square(img_a)+np.square(img_b))
    sigma_c = np.std(chroma)

    # item-2
    img_lum = img_lum.flatten()
    sorted_index = np.argsort(img_lum)
    top_index = sorted_index[int(len(img_lum)*0.99)]
    bottom_index = sorted_index[int(len(img_lum)*0.01)]
    con_lum = img_lum[top_index] - img_lum[bottom_index]

    # item-3
    chroma = chroma.flatten()
    sat = np.divide(chroma, img_lum, out=np.zeros_like(chroma, dtype=np.float64), where=img_lum!=0)
    avg_sat = np.mean(sat)

    uciqe = sigma_c*coe_Metric[0] + con_lum*coe_Metric[1] + avg_sat*coe_Metric[2]
    return uciqe

#### ***PSNR***

In [15]:
def MSE(img1,img2):
    diff = np.float64((img1-img2) ** 2)
    mse = np.mean(diff)
    if mse == 0:
        return 100
    return mse


def psnr(img1,img2):
    diff = np.float64((img1-img2) ** 2)
    mse = np.mean(diff)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX/np.sqrt(mse))

#### ***SSIM***

In [16]:
def getSSIM(X, Y):
    assert (X.shape == Y.shape), "Image-patche provided have different dimensions"
    nch = 1 if X.ndim==2 else X.shape[-1]
    mssim = []
    for ch in range(nch):
        Xc, Yc = X[...,ch].astype(np.float64), Y[...,ch].astype(np.float64)
        mssim.append(compute_ssim(Xc, Yc))
    return np.mean(mssim)


def compute_ssim(X, Y):
    # variables are initialized as suggested in the paper
    K1 = 0.01
    K2 = 0.03
    sigma = 1.5
    win_size = 5   

    # means
    ux = gaussian_filter(X, sigma)
    uy = gaussian_filter(Y, sigma)

    # variances and covariances
    uxx = gaussian_filter(X * X, sigma)
    uyy = gaussian_filter(Y * Y, sigma)
    uxy = gaussian_filter(X * Y, sigma)

    # normalize by unbiased estimate of std dev 
    N = win_size ** X.ndim
    unbiased_norm = N / (N - 1)  # eq. 4 of the paper
    vx  = (uxx - ux * ux) * unbiased_norm
    vy  = (uyy - uy * uy) * unbiased_norm
    vxy = (uxy - ux * uy) * unbiased_norm

    R = 255
    C1 = (K1 * R) ** 2
    C2 = (K2 * R) ** 2
    # compute SSIM (eq. 13 of the paper)
    sim = (2 * ux * uy + C1) * (2 * vxy + C2)
    D = (ux ** 2 + uy ** 2 + C1) * (vx + vy + C2)
    SSIM = sim/D 
    mssim = SSIM.mean()

    return mssim

#### ***Skimagemetrics PSNR SSIM***

In [17]:
def getPSNR(im_true,im_test):
    return peak_signal_noise_ratio(im_true,im_test)

def SSIMM(im_true,im_test):
    return structural_similarity(im_true,im_test,  multichannel=True)